# Dự báo GDP Việt Nam sử dụng mô hình ARIMA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 1. ĐỌC VÀ TIỀN XỬ LÝ DỮ LIỆU
# ---------------------------------------------------------
df = pd.read_csv('GDPcompare.csv')
# Lọc lấy dữ liệu Việt Nam và bỏ các giá trị rỗng (NaN)
data = df[['Year', 'Vietnam']].dropna()

# Chuyển Year thành index cho time series
data['Year'] = data['Year'].astype(int)
data.set_index('Year', inplace=True)

print("Dữ liệu GDP Việt Nam:")
print(data.head())
print(f"\nTổng số năm: {len(data)}")

In [ ]:
# 2. CHIA TẬP DỮ LIỆU (TRAIN/TEST SPLIT)
# ---------------------------------------------------------
# Để đánh giá chính xác, "giấu" đi 5 năm cuối làm tập Test
n_test = 5
train = data[:-n_test]  # Dữ liệu từ 1985 đến 2010
test = data[-n_test:]   # Dữ liệu từ 2011 đến 2015 (để kiểm tra)

print(f"Dữ liệu Huấn luyện: {len(train)} năm ({train.index[0]} - {train.index[-1]})")
print(f"Dữ liệu Kiểm thử:   {len(test)} năm ({test.index[0]} - {test.index[-1]})")

In [ ]:
# 3. HUẤN LUYỆN & ĐÁNH GIÁ MÔ HÌNH ARIMA
# ---------------------------------------------------------
# Xây dựng mô hình ARIMA trên tập Train
# ARIMA(p,d,q) - p: autoregressive, d: differencing, q: moving average
# Thử nghiệm với ARIMA(1,1,1) - phổ biến cho dữ liệu kinh tế
model = ARIMA(train['Vietnam'], order=(1, 1, 1))
model_fit = model.fit()

# Dự báo trên tập Test
forecast = model_fit.forecast(steps=len(test))
y_pred = forecast.values

# Tính toán các chỉ số đánh giá sai số
mae = mean_absolute_error(test['Vietnam'], y_pred)
rmse = np.sqrt(mean_squared_error(test['Vietnam'], y_pred))

print("--- KẾT QUẢ ĐÁNH GIÁ ĐỘ CHÍNH XÁC (RMSE, MAE) ---")
print(f"MAE (Sai số tuyệt đối trung bình):   {mae:,.0f} USD")
print(f"RMSE (Sai số căn bậc hai trung bình): {rmse:,.0f} USD")

# In ra dự báo so với thực tế
print("\n--- SO SÁNH DỰ BÁO VS THỰC TẾ ---")
for year, actual, pred in zip(test.index, test['Vietnam'], y_pred):
    print(f"Năm {year}: Thực tế = {actual:,.0f} USD, Dự báo = {pred:,.0f} USD")

In [ ]:
# 4. THỰC NGHIỆM DỰ BÁO TƯƠNG LAI
# ---------------------------------------------------------
# Sau khi đánh giá xong, train lại mô hình trên TOÀN BỘ dữ liệu
# để có kết quả dự báo tốt nhất cho tương lai.
final_model = ARIMA(data['Vietnam'], order=(1, 1, 1))
final_model_fit = final_model.fit()

# Dự báo cho các năm tiếp theo (2016 - 2025)
future_steps = 10
future_forecast = final_model_fit.forecast(steps=future_steps)

# Tạo index cho năm tương lai
last_year = data.index[-1]
future_years = range(last_year + 1, last_year + future_steps + 1)

print("--- DỰ BÁO TĂNG TRƯỞNG TRUNG HẠN (2016 - 2025) ---")
for year, val in zip(future_years, future_forecast):
    print(f"Năm {year}: {val:,.0f} USD")

In [ ]:
# 5. VẼ BIỂU ĐỒ SO SÁNH
# ---------------------------------------------------------
plt.figure(figsize=(14, 8))

# Vẽ dữ liệu thực tế
plt.plot(data.index, data['Vietnam'], 'bo-', label='Dữ liệu Thực tế (Historical Data)', linewidth=2)

# Vẽ dự báo trên tập test
plt.plot(test.index, y_pred, 'ro--', label='Dự báo Test (2011-2015)', linewidth=2, markersize=8)

# Vẽ dự báo tương lai
plt.plot(future_years, future_forecast, 'go--', label='Dự báo Tương lai (2016-2025)', linewidth=2, markersize=8)

plt.title('Dự báo GDP Việt Nam: Mô hình ARIMA(1,1,1)', fontsize=16, fontweight='bold')
plt.xlabel('Năm', fontsize=12)
plt.ylabel('GDP (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.ticklabel_format(style='plain', axis='y')  # Hiển thị số đầy đủ thay vì 1e11

# Thêm annotation cho các điểm dự báo
for year, val in zip(future_years, future_forecast):
    plt.annotate(f'{val:,.0f}', (year, val), textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()